## Work
1. 試比較 save_best_only 與否的差異
2. 請僅存入將 save_weights_only 設定為 True, 並嘗試 reset ipynb 並將模型與權重重新建回並預測 x_test

In [1]:
import os
import keras
import tensorflow
from keras.datasets import cifar10

import matplotlib.pyplot as plt
%matplotlib inline


# 本作業可以不需使用 GPU, 將 GPU 設定為 "無" (若想使用可自行開啟)
os.environ["CUDA_VISIBLE_DEVICES"] = ""

2021-11-22 17:24:16.704061: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-22 17:24:16.704096: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
num_classes = 10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.np_utils.to_categorical(y, num_classes)
    return y    

In [4]:
# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:

from keras.layers import BatchNormalization

"""
建立神經網路，並加入 BN layer
"""
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(input_layer)
            x = BatchNormalization()(x)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(x)
            x = BatchNormalization()(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [6]:
## 超參數設定
LEARNING_RATE = 1e-3
EPOCHS = 50
BATCH_SIZE = 1024
MOMENTUM = 0.95

In [7]:
# 載入 Callbacks
"""Code Here
設定 callbacks: model checkpoint
"""
from keras.callbacks import ModelCheckpoint

model_ckpt = ModelCheckpoint(filepath="./tmp.h5", 
                             monitor="val_loss", 
                             save_best_only=True)




In [9]:
model = build_mlp(input_shape=x_train.shape[1:])
model.summary()
optimizer = keras.optimizers.gradient_descent_v2.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

model.fit(x_train, y_train, 
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          validation_data=(x_test, y_test), 
          shuffle=True,
          callbacks=[model_ckpt]
         )

model.save("final_model.h5")
model.save_weights("model_weights.h5")

# Collect results
train_loss = model.history.history["loss"]
valid_loss = model.history.history["val_loss"]
train_acc = model.history.history["accuracy"]
valid_acc = model.history.history["val_accuracy"]

pred_final = model.predict(x_test)

# Load back
model = keras.models.load_model("./tmp.h5")
pred_loadback = model.predict(x_test)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 3072)]            0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
batch_normalization_3 (Batch (None, 512)               2048      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
batch_normalization_4 (Batch (None, 256)               1024      
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
batch_normalization_5 (Batch (None, 128)               512 

2021-11-22 17:38:48.810993: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 614400000 exceeds 10% of free system memory.


Epoch 1/50
49/49 [==============================] - 4s 68ms/step - loss: 2.2085 - accuracy: 0.2653 - val_loss: 2.1377 - val_accuracy: 0.1889
Epoch 2/50
49/49 [==============================] - 3s 62ms/step - loss: 1.7408 - accuracy: 0.3918 - val_loss: 1.9362 - val_accuracy: 0.3148
Epoch 3/50
49/49 [==============================] - 3s 65ms/step - loss: 1.6180 - accuracy: 0.4329 - val_loss: 1.7960 - val_accuracy: 0.3862
Epoch 4/50
49/49 [==============================] - 3s 58ms/step - loss: 1.5465 - accuracy: 0.4584 - val_loss: 1.6971 - val_accuracy: 0.4199
Epoch 5/50
49/49 [==============================] - 3s 57ms/step - loss: 1.4930 - accuracy: 0.4796 - val_loss: 1.6248 - val_accuracy: 0.4385
Epoch 6/50
49/49 [==============================] - 3s 61ms/step - loss: 1.4469 - accuracy: 0.4967 - val_loss: 1.5901 - val_accuracy: 0.4425
Epoch 7/50
49/49 [==============================] - 3s 56ms/step - loss: 1.4075 - accuracy: 0.5120 - val_loss: 1.5428 - val_accuracy: 0.4573
Epoch 8/50
49

In [10]:
from sklearn.metrics import accuracy_score

final_model_acc = accuracy_score(y_true=y_test.argmax(axis=-1), y_pred=pred_final.argmax(axis=-1))
loadback_acc = accuracy_score(y_true=y_test.argmax(axis=-1), y_pred=pred_loadback.argmax(axis=-1))

print(f'Accuracy of final weights: {final_model_acc}')
print(f'Accuracy of best(loadback) weights: {loadback_acc}')


Accuracy of final weights: 0.4953
Accuracy of best(loadback) weights: 0.4981


## 載入 Callbacks : save_best_only=False

In [11]:
# 載入 Callbacks
"""Code Here
設定 callbacks: model checkpoint
"""
from keras.callbacks import ModelCheckpoint

model_ckpt = ModelCheckpoint(filepath="./tmp.h5", 
                             monitor="val_loss", 
                             save_best_only=False)

In [12]:
model = build_mlp(input_shape=x_train.shape[1:])
model.summary()
optimizer = keras.optimizers.gradient_descent_v2.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

model.fit(x_train, y_train, 
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          validation_data=(x_test, y_test), 
          shuffle=True,
          callbacks=[model_ckpt]
         )

model.save("final_model.h5")
model.save_weights("model_weights.h5")

# Collect results
train_loss = model.history.history["loss"]
valid_loss = model.history.history["val_loss"]
train_acc = model.history.history["accuracy"]
valid_acc = model.history.history["val_accuracy"]

pred_final = model.predict(x_test)

# Load back
model = keras.models.load_model("./tmp.h5")
pred_loadback = model.predict(x_test)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 3072)]            0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
batch_normalization_6 (Batch (None, 512)               2048      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
batch_normalization_7 (Batch (None, 256)               1024      
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
batch_normalization_8 (Batch (None, 128)               512 

/home/wanzi/ml/cupoyMarathon/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
2021-11-22 20:07:35.649218: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 614400000 exceeds 10% of free system memory.


Epoch 1/50
49/49 [==============================] - 3s 57ms/step - loss: 2.1788 - accuracy: 0.2729 - val_loss: 2.1621 - val_accuracy: 0.2104
Epoch 2/50
49/49 [==============================] - 2s 45ms/step - loss: 1.7551 - accuracy: 0.3881 - val_loss: 1.9729 - val_accuracy: 0.3061
Epoch 3/50
49/49 [==============================] - 2s 46ms/step - loss: 1.6243 - accuracy: 0.4311 - val_loss: 1.8324 - val_accuracy: 0.3619
Epoch 4/50
49/49 [==============================] - ETA: 0s - loss: 1.5471 - accuracy: 0.45 - 2s 46ms/step - loss: 1.5474 - accuracy: 0.4593 - val_loss: 1.7028 - val_accuracy: 0.4138
Epoch 5/50
49/49 [==============================] - 2s 45ms/step - loss: 1.4895 - accuracy: 0.4791 - val_loss: 1.6323 - val_accuracy: 0.4288
Epoch 6/50
49/49 [==============================] - 2s 45ms/step - loss: 1.4428 - accuracy: 0.4966 - val_loss: 1.5874 - val_accuracy: 0.4425
Epoch 7/50
49/49 [==============================] - 2s 51ms/step - loss: 1.4033 - accuracy: 0.5090 - val_loss: 1

In [ ]:
final_model_acc = accuracy_score(y_true=y_test.argmax(axis=-1), y_pred=pred_final.argmax(axis=-1))
loadback_acc = accuracy_score(y_true=y_test.argmax(axis=-1), y_pred=pred_loadback.argmax(axis=-1))

print(f'Accuracy of final weights: {final_model_acc}')
print(f'Accuracy of best(loadback) weights: {loadback_acc}')
